In [ ]:
#This eaxmple was given by Microsft Copilot
!pip install transformers datasets

In [ ]:
from datasets import load_dataset
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

# Load the dataset
dataset = load_dataset('imdb')

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True)


In [ ]:
# Freeze all the layers in the pre-trained model
for param in model.base_model.parameters():
    param.requires_grad = False

# Add adaptive layers
import torch.nn as nn

class AdaptiveModel(nn.Module):
    def __init__(self, model):
        super(AdaptiveModel, self).__init__()
        self.bert = model.bert
        self.classifier = nn.Sequential(
            nn.Linear(model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, model.config.num_labels)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return (loss, logits) if loss is not None else logits

adaptive_model = AdaptiveModel(model)


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define trainer
trainer = Trainer(
    model=adaptive_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer
)

# Train the model
trainer.train() #This one was still going on for 25 minutes when I downloaded this file


In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")
